In [1]:
import torch
from torch.utils.data import DataLoader, random_split
from simple_dataset import SimpleDataset, simple_audio_preprocess
import numpy as np
from transforms import Compose, RandomApply, Dequantize, RandomCrop
from core import random_phase_mangle
import matplotlib.pyplot as plt
import IPython.display as ipd
import os
import lmdb

In [2]:
SR = 48000
N_SIGNAL = 65536
num_workers = 0 if os.name == "nt" else 10
print(num_workers)

preprocess = lambda name: simple_audio_preprocess(
        SR,
        2 * N_SIGNAL,
    )(name).astype(np.float16)

dataset = SimpleDataset(
        SR,
        "test",
        "../speech-small/",
        preprocess_function=preprocess,
        split_set="full",
        transforms=Compose([
            lambda x: x.astype(np.float32),
            RandomCrop(N_SIGNAL),
            RandomApply(
                lambda x: random_phase_mangle(x, 20, 2000, .99, SR),
                p=.8,
            ),
            Dequantize(16),
            lambda x: x.astype(np.float32),
        ]),
    )

val = max((2 * len(dataset)) // 100, 1)
train = len(dataset) - val
train, val = random_split(
        dataset,
        [train, val],
        generator=torch.Generator().manual_seed(42),
    )

train = DataLoader(train, 8, True, drop_last=True, num_workers=num_workers)

example = next(iter(train))

clean_example =example['data_clean'][0]
perturbed_example = example['data_perturbed'][0]

print(clean_example.shape, perturbed_example.shape)

ipd.Audio(perturbed_example, rate=SR, normalize=True)

10


Error: Attempt to operate on closed/deleted/dropped object.

In [ ]:
ipd.Audio(clean_example, rate=SR, normalize=True)